In [38]:
import csv
import json
from copy import deepcopy
import ast
csv.field_size_limit(1131072)

1131072

In [39]:
def remove_drop_words(alignment):
    result = set()
    for start, end in alignment:
        result.add((start, end//2))
    return result

def extract_pair(result):
    starts = set()
    ends = set()
    for start, end in result:
        starts.add(start)
        ends.add(end)
    start_idx = min(list(starts))
    end_idx = max(list(ends))
    return [min(list(ends)), max(list(ends))+1]

def extract_alignment_pair_original(alignment):
    result = set()
    for s in range(len(alignment)):
        for t in range(len(alignment[0])):
            if alignment[s][t]:
                result.add((s, t))
    return sorted(list(result))

def extract_alignment_pair_space(alignment):
    result = set()
    for s in range(len(alignment)):
        for t in range(len(alignment[0])):
            if alignment[s][t]:
                result.add((s, t))
    return sorted(list(remove_drop_words(result)))

In [11]:
source_data = []
with open('data/fa/source/to_tasa_all_fa_test_with_id.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for item in reader:
        source_data.append(item)

In [12]:
print(len(source_data))
print(source_data[0].keys())
print(source_data[0])

7704
dict_keys(['src_tokens', 'tar_tokens', 'config_obj', 'mention_id'])
{'src_tokens': '[\'There\', \'you\', \'go\', \'.\', \'You\', "\'re", \'all\', \'done\', \'!\']', 'tar_tokens': "['ردیف', 'شد']", 'config_obj': '{"src_enable_retokenize": false, "version": {"PATCH": 0, "MAJOR": 1, "MINOR": 0}, "tar_enable_retokenize": false,"src_head_inds":[0],"alignment":[[true, false], [false, false], [false, false], [false, false], [false, false], [false, false], [false, false], [false, false], [false, false]]}', 'mention_id': 's09e03c09t|0'}


In [13]:
source_inputs = []
for set_key in ["0_1000", "1000_2000", "2000_3000", "3000_4000","4000_5000", "5000_6000","6000_7000", "7000_end"]:
    with open('data/fa/inputs/to_tasa_fa_drop_test_set_'+set_key+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for item in reader:
            source_inputs.append(item)

In [14]:
print(len(source_inputs))
print(source_inputs[0].keys())
print(source_inputs[0])

7704
dict_keys(['src_tokens', 'tar_tokens', 'config_obj'])
{'src_tokens': '[\'There\', \'you\', \'go\', \'.\', \'You\', "\'re", \'all\', \'done\', \'!\']', 'tar_tokens': '["\\u0631\\u062f\\u06cc\\u0641", "", "\\u0634\\u062f", ""]', 'config_obj': '{"src_enable_retokenize": false, "version": {"PATCH": 0, "MAJOR": 1, "MINOR": 0}, "tar_enable_retokenize": false,"src_head_inds":[0],"alignment":[[true, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false], [false, false, false, false]]}'}


In [15]:
coe_annotations = []
# Batch 0: first 400
for split_name in ["batch_0_0_448"]:
    with open('data/fa/annotations/'+split_name+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for i, item in enumerate(reader):
            if i>=400:
                continue
            coe_annotations.append(item)
# Batch 0: later 600
for split_name in ["batch_0_400_1000"]:
    with open('data/fa/annotations/'+split_name+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for item in reader:
            coe_annotations.append(item)

for split_name in ["batch_1", "batch_2","batch_3","batch_4","batch_5","batch_6", "batch_7"]:
    with open('data/fa/annotations/'+split_name+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for item in reader:
            coe_annotations.append(item)

In [16]:
# coe_annotations = []
# # for split_name in ["batch_0_0_448"]:
# for split_name in ["batch_0_400_1000"]:
#     with open('data/fa/annotations/'+split_name+'.csv', 'r', encoding='utf-8') as f:
#         reader = csv.DictReader(f)
#         for item in reader:
#             coe_annotations.append(item)

In [17]:
all_data = {}
with open('data/fa/source/all_coref_data_en_finalized.json', 'r') as f:
    temp = json.load(f)
    for item in temp:
        all_data[item['scene_id']] = item

mention_dict = {}
for item in all_data:
    annotations = all_data[item]['annotations']
    for anno in annotations:
        query = anno['query']
        antecedents = anno['antecedents']
        mention_dict[query['mention_id']] = [query['sentenceIndex'], query['startToken'],query['endToken']]
        if antecedents!=[]:
            if isinstance(antecedents[0], dict):
                for ante in antecedents:
                    mention_dict[ante['mention_id']] = [ante['sentenceIndex'], ante['startToken'],ante['endToken']]

In [18]:
all_data_fa = {}
with open('data/fa/source/all_coref_data_en_fa_finalized.json', 'r') as f:
    temp = json.load(f)
    for item in temp:
        all_data_fa[item['scene_id']] = item

mention_dict_fa = {}
for item in all_data_fa:
    annotations = all_data_fa[item]['annotations']
    for anno in annotations:
        query = anno['query']
        antecedents = anno['antecedents']
        mention_dict_fa[query['mention_id']] = [query['sentenceIndex'], query['startToken'],query['endToken']]
        if antecedents!=[]:
            if isinstance(antecedents[0], dict):
                for ante in antecedents:
                    mention_dict_fa[ante['mention_id']] = [ante['sentenceIndex'], ante['startToken'],ante['endToken']]

In [19]:
print(ast.literal_eval(coe_annotations[1]['Input.src_tokens']))
print(type(ast.literal_eval(coe_annotations[1]['Input.src_tokens'])))

['There', 'you', 'go', '.', 'You', "'re", 'all', 'done', '!']
<class 'list'>


In [30]:
correction_dict = {}
remove_set = set()
add_dict = {}

all_correction_lines = []

# for i in range(len(coe_annotations)):
count = 0
# for i in range(6400):
for i in range(len(coe_annotations)):
    # extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment'])
    # extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment'])

    # if source_data[i]['mention_id'] in mention_dict_fa:
    #     temp_0 = extract_pair(extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment']))
    #     temp_1 = extract_pair(extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment']))
    #     sent_id, start, end = mention_dict_fa[source_data[i]['mention_id']]
    #     mention_text_source = "".join(ast.literal_eval(source_data[i]['tar_tokens'])[temp_0[0]: temp_0[1]])
    #     mention_text_tasa = "".join(all_data_fa[source_data[i]['mention_id'][:10]]['sentences'][sent_id][start: end])
    #     print(source_data[i]['mention_id'], source_data[i]['mention_id'][:10], [sent_id, start, end], temp_0, temp_1)
    #     print(mention_text_source==mention_text_tasa, mention_text_source, mention_text_tasa)
    #     print("".join(all_data_fa[source_data[i]['mention_id'][:10]]['sentences'][sent_id]))
    #     # print(extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment']))
    #     # print(extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment']))
    #     print()
    #     # if mention_text_source!=mention_text_tasa:
    #     #     print(mention_text_source==mention_text_tasa, mention_text_source, mention_text_tasa)
    #     # # print()

    original_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Input.config_obj'])['alignment'])
    corrected_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Answer.alignment']))
    if original_result==corrected_result:
        continue
    mention_id = source_data[i]['mention_id']

    a = ()
    b = ()
    c = ()
    if original_result!=[]:
        a = extract_pair(original_result)
    if corrected_result!=[]:
        b = extract_pair(corrected_result)
    if mention_id in mention_dict_fa:
        c = [mention_dict_fa[mention_id][1], mention_dict_fa[mention_id][2]]
    # print(a==c, mention_id, a, c, b)

    # sentence = ast.literal_eval(coe_annotations[i]['Input.tar_tokens'])
    sentence = ast.literal_eval(source_data[i]['tar_tokens'])
    original_text = ""
    corrected_text = ""
    if a!=():
        original_text = " ".join(sentence[a[0]: a[1]])
    if b!=():
        corrected_text = " ".join(sentence[b[0]: b[1]])
    print(mention_id)
    print("Original Mention Index:", a, original_text)
    print("Corrected Mention Index:", b, corrected_text)
    count += 1
    # if a==() and b!=():
    #     print("Add")
    #     count += 1
    # if a!=() and b==():
    #     print("Delete")
    #     count += 1
    print()
    all_correction_lines.append([
        mention_id,
        a,
        b,
        original_text,
        corrected_text
    ])
    # print(source_data[i]['mention_id'], a, b)
    # print(mention_dict[mention_id], a)
    # print()

    # if source_data[i]['mention_id'] in mention_dict:
    #     mention_id = source_data[i]['mention_id']
    #     target = mention_dict[source_data[i]['mention_id']][1:]
    #     correction_dict[mention_id] = [a, b]
    if b==(): # Add mention_id to remove set
        remove_set.add(source_data[i]['mention_id'])
    elif a==(): # Collect mentions to add
        add_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]
    else:
        correction_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]

s09e03c09t|0
Original Mention Index: [0, 1] ردیف
Corrected Mention Index: () 

s09e03c09t|6
Original Mention Index: () 
Corrected Mention Index: [3, 4] نبود

s09e03c09t|7
Original Mention Index: () 
Corrected Mention Index: [6, 7] مامانم

s09e03c09t|10
Original Mention Index: () 
Corrected Mention Index: [9, 10] ببینتم

s09e03c09t|12
Original Mention Index: () 
Corrected Mention Index: [14, 15] میکرد

s09e03c09t|14
Original Mention Index: () 
Corrected Mention Index: [13, 14] زندانیم

s09e03c09t|17
Original Mention Index: () 
Corrected Mention Index: [1, 2] بود

s09e03c09t|23
Original Mention Index: () 
Corrected Mention Index: [13, 14] میتونستم

s09e03c09t|26
Original Mention Index: [10, 11] شکاف
Corrected Mention Index: [10, 13] شکاف ‌ ها

s09e03c09t|27
Original Mention Index: () 
Corrected Mention Index: [3, 4] دارم

s09e03c09t|34
Original Mention Index: () 
Corrected Mention Index: [10, 11] مامانت

s09e03c09t|38
Original Mention Index: () 
Corrected Mention Index: [10, 11] میگفت

s

In [31]:
print(count, len(source_inputs), count/len(source_inputs)*100)

2485 7704 32.25597092419522


In [37]:
# print((187)/2485*100)
# print((1504)/2485*100)
# print((2485-1504-187)/2485*100)
print((187)/len(source_inputs)*100)
print((1504)/len(source_inputs)*100)
print((2485-1504-187)/len(source_inputs)*100)

2.427310488058152
19.52232606438214
10.306334371754932


In [11]:
print(count/len(coe_annotations), count, len(coe_annotations))

0.3225597092419522 2485 7704


In [104]:
print(['اونقدرا', '', 'هم', '', 'بد', '', 'نبود', '', 'خشگله', '', 'اگه', '', 'مامانم', '', 'میتونست', '', 'الان', '', 'ببینتم', '', 'توی', '', 'کمد', '', 'گناهکارا', '', 'زندانیم', '', 'میکرد', ''][3: 4])

['']


In [15]:
import pickle as pkl

scene_ids = set()
for item in correction_dict:
    scene_ids.add(item[:10])
for item in remove_set:
    scene_ids.add(item[:10])
for item in add_dict:
    scene_ids.add(item[:10])

all_results = {}
for item in scene_ids:
    all_results[item] = {
        "correction_dict": {},
        "remove_set": set(),
        "add_dict": {}
    }

for item in remove_set:
    all_results[item[:10]]['remove_set'].add(item)
for item in correction_dict:
    all_results[item[:10]]['correction_dict'][item] = correction_dict[item]
for item in add_dict:
    all_results[item[:10]]['add_dict'][item] = add_dict[item]

with open('data/fa/correction_results.pkl', 'wb') as f:
    pkl.dump(all_results, f)

In [66]:
names = ['correction_dict', 'remove_set', 'add_dict']
for scene_id in all_results:
    print(scene_id)
    for name in names:
        print(name, all_results[scene_id][name])
    print()

s09e10c05t
correction_dict {'s09e10c05t|0': [0, 6, 8], 's09e10c05t|2': [0, 3, 4], 's09e10c05t|4': [0, 15, 22], 's09e10c05t|19': [3, 6, 7], 's09e10c05t|26': [4, 0, 7], 's09e10c05t|53': [6, 33, 36], 's09e10c05t|54': [6, 33, 39], 's09e10c05t|58': [6, 49, 56], 's09e10c05t|59': [6, 51, 55]}
remove_set {'s09e10c05t|8', 's09e10c05t|7'}
add_dict {'s09e10c05t|3': [0, 12, 13], 's09e10c05t|28': [4, 0, 1], 's09e10c05t|40': [6, 12, 13], 's09e10c05t|44': [6, 19, 20], 's09e10c05t|49': [6, 32, 33], 's09e10c05t|55': [6, 38, 39], 's09e10c05t|65': [7, 6, 7], 's09e10c05t|78': [8, 4, 5]}

s09e16c04t
correction_dict {}
remove_set set()
add_dict {'s09e16c04t|0': [0, 2, 3], 's09e16c04t|6': [0, 7, 8], 's09e16c04t|9': [1, 5, 6], 's09e16c04t|13': [3, 0, 1], 's09e16c04t|19': [6, 1, 2], 's09e16c04t|25': [7, 2, 3], 's09e16c04t|32': [8, 9, 10], 's09e16c04t|41': [9, 3, 4], 's09e16c04t|51': [10, 11, 12]}

s09e11c08t
correction_dict {'s09e11c08t|22': [4, 7, 11], 's09e11c08t|25': [5, 3, 7], 's09e11c08t|60': [10, 4, 12],

In [14]:
# Write correction results to csv files
with open('farsi_correction_results.csv', 'w', encoding="utf-8") as csv_w:
    fieldnames = ['Mention ID', 'Original Mention Index', 'Corrected Mention Index', 'Original Mention Text', 'Corrected Mention Text']
    writer = csv.DictWriter(csv_w, fieldnames, lineterminator='\n', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for line in all_correction_lines:
        writer.writerow({
            fieldnames[0]: line[0],
            fieldnames[1]: line[1],
            fieldnames[2]: line[2],
            fieldnames[3]: line[3],
            fieldnames[4]: line[4],
        })

In [13]:
print(all_correction_lines[0])

['s09e03c09t|0', [0, 1], (), 'ردیف', '']


In [119]:
print(all_results)

{'s09e09c07t': {'correction_dict': {'s09e09c07t|50': [8, 16, 19], 's09e09c07t|97': [13, 1, 3], 's09e09c07t|104': [13, 10, 13], 's09e09c07t|111': [14, 7, 9], 's09e09c07t|117': [15, 9, 12]}, 'remove_set': {'s09e09c07t|56', 's09e09c07t|31', 's09e09c07t|23', 's09e09c07t|5', 's09e09c07t|38', 's09e09c07t|39'}, 'add_dict': {'s09e09c07t|0': [0, 2, 3], 's09e09c07t|41': [8, 2, 4], 's09e09c07t|63': [8, 45, 46], 's09e09c07t|78': [10, 1, 2], 's09e09c07t|92': [11, 7, 8], 's09e09c07t|98': [13, 5, 6], 's09e09c07t|102': [13, 9, 10], 's09e09c07t|105': [13, 10, 11], 's09e09c07t|115': [15, 9, 10]}}, 's09e04c07t': {'correction_dict': {'s09e04c07t|13': [2, 15, 18], 's09e04c07t|17': [2, 9, 14], 's09e04c07t|38': [6, 14, 17]}, 'remove_set': {'s09e04c07t|70', 's09e04c07t|67'}, 'add_dict': {'s09e04c07t|11': [2, 9, 11], 's09e04c07t|27': [5, 7, 8], 's09e04c07t|36': [6, 14, 15], 's09e04c07t|57': [6, 30, 31], 's09e04c07t|83': [9, 13, 15]}}, 's09e10c04t': {'correction_dict': {'s09e10c04t|15': [3, 1, 2], 's09e10c04t|1

In [8]:
print(all_data.keys())
print(len(all_data))

dict_keys(['s01e01c00t', 's01e01c01t', 's01e01c03t', 's01e01c04t', 's01e01c05t', 's01e01c06t', 's01e01c07t', 's01e01c08t', 's01e01c09t', 's01e01c10t', 's01e03c00t', 's01e10c04t', 's01e03c01t', 's01e03c02t', 's01e03c03t', 's01e03c04t', 's01e03c05t', 's01e03c06t', 's01e03c07t', 's01e03c08t', 's01e03c09t', 's01e04c00t', 's01e04c01t', 's01e04c02t', 's01e04c03t', 's01e04c04t', 's01e04c05t', 's01e04c06t', 's01e04c07t', 's01e04c08t', 's01e04c09t', 's01e04c10t', 's01e04c11t', 's01e04c12t', 's01e05c00t', 's01e05c01t', 's01e05c02t', 's01e05c03t', '', 's01e05c05t', 's01e05c06t', 's01e05c07t', 's01e07c01t', 's01e07c02t', 's01e07c03t', 's01e07c04t', 's01e07c05t', 's01e09c00t', 's01e09c01t', 's01e09c02t', 's01e09c03t', 's01e09c04t', 's01e09c05t', 's01e09c06t', 's01e07c00t', 's01e09c07t', 's01e10c00t', 's01e10c01t', 's01e10c02t', 's01e10c03t', 's01e10c05t', 's01e10c07t', 's01e11c00t', 's01e11c01t', 's01e11c02t', 's01e11c03t', 's01e11c04t', 's01e11c05t', 's01e11c06t', 's01e11c07t', 's01e11c08t', 's04e

In [31]:
target = set()
for item in all_data.keys():
    if item[9]=="t":
        target.add(item[:6])
temp = [(int(item[1:3]), int(item[4:6])) for item in target]
print(sorted(list(target)))
print(len(target))
print(temp)

['s01e01', 's01e03', 's01e04', 's01e05', 's01e07', 's01e09', 's01e10', 's01e11', 's01e13', 's01e14', 's01e15', 's01e16', 's02e04', 's02e06', 's03e21', 's03e23', 's04e01', 's04e03', 's04e04', 's04e05', 's04e06', 's04e08', 's04e19', 's04e21', 's04e22', 's04e23', 's04e24', 's05e02', 's05e06', 's05e12', 's05e14', 's05e15', 's05e20', 's05e23', 's06e02', 's06e03', 's06e04', 's06e05', 's06e06', 's06e07', 's06e08', 's06e09', 's06e14', 's06e15', 's06e16', 's06e17', 's06e19', 's06e20', 's06e21', 's07e01', 's07e02', 's07e06', 's07e10', 's07e11', 's07e15', 's07e16', 's07e17', 's07e18', 's07e19', 's07e20', 's07e22', 's07e23', 's08e01', 's08e02', 's08e03', 's08e06', 's08e09', 's08e11', 's08e12', 's08e15', 's08e16', 's08e17', 's08e18', 's08e19', 's08e20', 's08e21', 's08e22', 's08e23', 's08e24', 's09e01', 's09e03', 's09e04', 's09e07', 's09e09', 's09e10', 's09e11', 's09e12', 's09e16']
88
[(7, 15), (6, 16), (7, 23), (5, 2), (9, 3), (5, 14), (1, 9), (6, 19), (7, 2), (1, 1), (8, 19), (3, 21), (7, 16), (8,

In [32]:
target = set()
for item in all_data.keys():
    if item[9]=="f":
        target.add(item[:6])
temp = [(int(item[1:3]), int(item[4:6])) for item in target]
print(sorted(list(target)))
print(len(target))
print(temp)

['s01e01', 's01e02', 's01e03', 's01e04', 's01e06', 's01e07', 's01e08', 's01e09', 's01e10', 's01e12', 's01e13', 's01e14', 's01e15', 's01e16', 's01e17', 's01e18', 's01e19', 's01e20', 's01e22', 's01e24', 's06e09']
21
[(1, 10), (1, 22), (1, 15), (1, 9), (1, 1), (1, 12), (1, 17), (1, 4), (1, 24), (1, 6), (1, 19), (1, 8), (1, 18), (1, 16), (1, 2), (6, 9), (1, 13), (1, 14), (1, 3), (1, 7), (1, 20)]


In [9]:
print(correction_dict.keys())

dict_keys(['s09e03c09t|0', 's09e03c09t|3', 's09e03c09t|17', 's09e03c09t|18', 's09e03c09t|25', 's09e03c09t|26', 's09e03c09t|62', 's09e03c09t|91', 's09e03c09t|103', 's09e03c09t|123', 's09e03c09t|141', 's09e03c09t|154', 's09e03c09t|178', 's09e03c09t|196', 's09e03c09t|197', 's09e03c09t|203', 's09e03c09t|217', 's09e03c09t|230', 's09e03c09t|234', 's09e03c09t|252', 's09e03c09t|267', 's09e03c09t|268', 's09e03c09t|270', 's09e03c10t|11', 's09e03c10t|12', 's09e03c10t|13', 's09e03c10t|16', 's09e03c10t|33', 's09e03c10t|34', 's09e03c10t|40', 's09e03c10t|41', 's09e03c10t|42', 's09e03c10t|43', 's09e03c10t|48', 's09e03c10t|57', 's09e03c10t|69', 's09e03c10t|70', 's09e03c10t|71', 's09e03c10t|72', 's09e03c11t|1', 's09e03c11t|10', 's09e03c11t|11', 's09e03c11t|14', 's09e03c11t|21', 's09e03c11t|28', 's09e03c11t|63', 's09e03c11t|71', 's09e03c12t|10', 's09e03c12t|28', 's09e03c12t|39', 's09e03c12t|49', 's09e03c12t|57', 's09e03c12t|58', 's09e03c12t|59', 's09e03c12t|88', 's09e03c12t|119', 's09e03c12t|120', 's09e0

In [10]:
print(all_data['s09e03c09t'])

{'sentences': [['就', '这', '样', '啦', '耳', '洞', '打', '好', '了', '!'], ['感', '觉', '挺', '不', '错', '的', '好', '漂', '亮', '天', '啊', '要', '是', '我', '妈', '妈', '看', '到', '我', '她', '要', '把', '我', '锁', '进', '罪', '孽', '柜', '里', '面', '壁', '思', '过', '了'], ['你', '在', '开', '玩', '笑', '吧', '?'], ['说', '实', '话', '她', '才', '是', '玩', '笑', '我', '还', '能', '从', '柜', '子', '缝', '里', '看', '电', '视', '呢'], ['我', '终', '于', '明', '白', '为', '什', '么', '你', '和', '你', '妈', '妈', '的', '关', '系', '不', '亲', '密', '了'], ['但', '是', '她', '要', '是', '结', '婚', '了', '她', '还', '是', '会', '告', '诉', '妈', '妈', '的'], ['你', '抓', '着', '这', '事', '不', '放', '手', '了', '吗', '?'], ['他', '们', '不', '知', '道', '这', '事', '让', '人', '很', '伤', '心', '我', '是', '说', '他', '们', '都', '爱', 'L', 'e', 'o', 'n', 'a', 'r', 'd'], ['好', '吧', '你', '们', '要', '我', '打', '电', '话', '吗', '?', '我', '打', '电', '话', '为', '什', '么', '我', '会', '这', '么', '紧', '张', '?'], ['这', '可', '以', '理', '解', 'A', 'm', 'y', '告', '诉', '妈', '妈', '她', '和', 'S', 'h', 'e', 'l', 'd', 'o', 'n', '分', '手', '

In [12]:
# Perform Correction
with open('data/zh/source/all_coref_data_en_zh_finalized.json', 'r') as f:
    source = json.load(f)

In [29]:
output = []
for i, scene in enumerate(source):
    if scene['scene_id'] != "s09e07c07t":
        continue
    new_annotations = []
    annotations = deepcopy(scene['annotations'])
    for item in annotations:
        query = deepcopy(item['query'])
        antecedents = deepcopy(item['antecedents'])
        # print(query, antecedents)
        # Correct Query
        if query['mention_id'] in correction_dict:
            print(query, correction_dict[query['mention_id']])

        new_annotations.append({
            "query": query,
            "antecedents": antecedents
        })



{'sentenceIndex': 1, 'startToken': 0, 'endToken': 1, 'mention_id': 's09e07c07t|2'} [[0, 1], [15, 16]]
{'sentenceIndex': 1, 'startToken': 43, 'endToken': 45, 'mention_id': 's09e07c07t|7'} [[43, 45], ()]
{'sentenceIndex': 2, 'startToken': 1, 'endToken': 4, 'mention_id': 's09e07c07t|8'} [[1, 4], ()]
{'sentenceIndex': 2, 'startToken': 4, 'endToken': 6, 'mention_id': 's09e07c07t|10'} [[4, 6], ()]
{'sentenceIndex': 4, 'startToken': 8, 'endToken': 10, 'mention_id': 's09e07c07t|13'} [[8, 10], ()]
{'sentenceIndex': 5, 'startToken': 8, 'endToken': 9, 'mention_id': 's09e07c07t|19'} [[8, 9], [17, 18]]
{'sentenceIndex': 7, 'startToken': 3, 'endToken': 4, 'mention_id': 's09e07c07t|30'} [[3, 4], ()]
{'sentenceIndex': 7, 'startToken': 3, 'endToken': 4, 'mention_id': 's09e07c07t|30'} [[3, 4], ()]
{'sentenceIndex': 7, 'startToken': 10, 'endToken': 12, 'mention_id': 's09e07c07t|33'} [[10, 12], ()]
{'sentenceIndex': 8, 'startToken': 3, 'endToken': 5, 'mention_id': 's09e07c07t|44'} [[3, 5], ()]
{'sentenceI

In [30]:
print(correction_dict['s09e07c07t|30'])

[[3, 4], ()]


In [31]:
print(all_data['s09e07c07t'])

{'sentences': [[], ['你', '在', '这', '里', '感', '觉', '还', '好', '嘛', '?', '没', '事', '挺', '好', '的', '我', '感', '觉', '自', '己', '像', '个', '考', '古', '学', '家', '像', '是', 'I', 'n', 'd', 'i', 'a', 'n', 'a', 'J', 'o', 'n', 'e', 's', '与', '独', '栋', '别', '墅', '嘿', '看', '啊', '!'], ['我', '捡', '了', '个', '贝', '壳'], [], ['其', '实', '是', '老', '鼠', '的', '骨', '头', '淡', '定'], ['这', '的', '灰', '尘', '够', '多', '的', '了', '你', '就', '别', '再', '折', '腾', '了', '抱', '歉', '你', '知', '道', '.', '.', '.', '这', '里', '是', '什', '么', '位', '置', '吗', '?'], ['E', 'M', 'P', 'T', 'Y'], ['大', '概', '吧', '我', '觉', '得', '那', '边', '是', '小', '客', '厅'], ['好', '吧', '那', '这', '里', '就', '是', '餐', '厅', '了', '没', '错'], ['不', '过', '把', '小', '客', '厅', '改', '造', '成', '婴', '儿', '室', '也', '不', '会', '很', '费', '事'], ['我', '为', '什', '么', '会', '不', '想', '听', '你', '说', '这', '件', '事', '?'], ['因', '为', 'B', 'e', 'r', 'n', 'i', 'e', '说', '你', '不', '想', '要', '孩', '子', '不', '是', '这', '样', '的', '我', '.', '.', '.', '我', '很', '希', '望', '她', '能', '怀', '上', '一', '个'

In [88]:
sample = all_data['s09e03c10t']
sentences = sample['sentences']
annotations = sample['annotations']
for sent in sentences:
    print(sent)

[]
['这', '个', '叫', '做', '拉', '玛', '泽', '呼', '吸', '可', '以', '帮', '你', '用', '力']
['那', '你', '用', '子', '宫', '推', '好', '了']
['应', '该', '有', '个', '更', '容', '易', '的', '办', '法', '把', '它', '弄', '下', '来', '的']
['是', '的', '这', '不', '过', '就', '是', '个', '物', '理', '学', '问', '题', '罢', '了']
['阿', '基', '米', '德', '曾', '经', '说', '了', '"', '给', '我', '一', '根', '足', '够', '长', '的', '的', '杠', '杆', '和', '一', '个', '支', '点', '我', '就', '能', '撬', '动', '地', '球', '"']
['我', '们', '只', '需', '要', '找', '点', '原', '材', '料', '把', '这', '个', '杠', '杆', '做', '出', '来', '就', '行', '了']
['一', '点', '不', '错', '我', '敢', '打', '赌', '这', '附', '近', '有', '东', '西', '能', '当', '这', '个', '"', '撬', '棍', '"']
['让', '我', '找', '找', '"', '撬', '棍', '"', '在', '哪', '呢', '.', '.', '.']
['大', '家', '等', '一', '下', '!', '我', '们', '按', '顺', '序', '一', '个', '个', '解', '决', '问', '题', '首', '先', '我', '们', '得', '决', '定', '我', '们', '是', '叫', '它', '杠', '杆', '还', '是', '撬', '棍', '等', '我', '们', '决', '定', '叫', '它', '杠', '杆', '之', '后', '就', '可', '以', '卷', '起', '袖', '子'

In [89]:
for item in annotations:
    print(item)
    print(sentences[item['query']['sentenceIndex']][item['query']['startToken']: item['query']['endToken']])
    print()

{'query': {'sentenceIndex': 1, 'startToken': 0, 'endToken': 2, 'mention_id': 's09e03c10t|3'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['这', '个']

{'query': {'sentenceIndex': 1, 'startToken': 12, 'endToken': 13, 'mention_id': 's09e03c10t|6'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['你']

{'query': {'sentenceIndex': 2, 'startToken': 1, 'endToken': 2, 'mention_id': 's09e03c10t|7'}, 'antecedents': ['Raj']}
['你']

{'query': {'sentenceIndex': 2, 'startToken': 3, 'endToken': 5, 'mention_id': 's09e03c10t|11'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['子', '宫']

{'query': {'sentenceIndex': 3, 'startToken': 2, 'endToken': 4, 'mention_id': 's09e03c10t|12'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['有', '个']

{'query': {'sentenceIndex': 3, 'startToken': 5, 'endToken': 13, 'mention_id': 's09e03c10t|13'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['容', '易', '的', '办', 